Joining master dataframes 2001-2021

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

Step 1: Upload institutional characteristics as reference dataframe
1. Upload 
2. Create unique ID for data merging: unitid_year

In [2]:
ic_master = pd.read_csv("institutional_characteristics_master.csv",low_memory=False,encoding = "ISO-8859-1")
ic_master['unitid_year'] = ic_master['unitid'].astype(str) + "_" + ic_master['year'].astype(str)
#ic_master.head(2)
ic_master.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61008 entries, 0 to 61007
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   unitid            61008 non-null  int64 
 1   instnm            61008 non-null  object
 2   addr              60686 non-null  object
 3   city              61008 non-null  object
 4   stabbr            61008 non-null  object
 5   zip               61008 non-null  object
 6   obereg            61008 non-null  object
 7   sector            61008 non-null  object
 8   iclevel           61008 non-null  object
 9   control           61008 non-null  object
 10  deggrant          61008 non-null  object
 11  carnegie          61008 non-null  object
 12  pset4flg          61008 non-null  object
 13  medical           61008 non-null  object
 14  hospital          61008 non-null  object
 15  openpubl          61008 non-null  object
 16  year              61008 non-null  int64 
 17  academic yea

Step 2: cross reference student enrollment & fte
1. Upload 
2. Create unique ID for data merging: unitid_year
3. left outer join with ic_master
4. check for missing values

In [3]:
student_master = pd.read_csv("student_enrollment_fte_master.csv",low_memory=False,encoding = "ISO-8859-1")
student_master['unitid_year'] = student_master['unitid'].astype(str) + "_" + student_master['year'].astype(str)
#student_master = student_master.rename(columns={'total fte':'total fte student'}) 
#student_master.head()
student_master_for_merge = student_master.drop(columns=['unitid','year'])
#student_master_for_merge.head()

student_master_for_merge.head()

student fte  unitid_year
0       5223.0  100654_2001
1      11614.0  100663_2001
2        306.0  100690_2001
3       4996.0  100706_2001
4       4804.0  100724_2001

Left outer join:
institutional characteristics + student enrollment & fte 

In [4]:
ic_student_join = pd.merge(ic_master,student_master_for_merge, left_on='unitid_year',right_on='unitid_year',how = 'left')
#ic_student_join['year'] = pd.to_datetime(ic_student_join.year, format='%Y')
ic_student_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61008 entries, 0 to 61007
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   unitid            61008 non-null  int64  
 1   instnm            61008 non-null  object 
 2   addr              60686 non-null  object 
 3   city              61008 non-null  object 
 4   stabbr            61008 non-null  object 
 5   zip               61008 non-null  object 
 6   obereg            61008 non-null  object 
 7   sector            61008 non-null  object 
 8   iclevel           61008 non-null  object 
 9   control           61008 non-null  object 
 10  deggrant          61008 non-null  object 
 11  carnegie          61008 non-null  object 
 12  pset4flg          61008 non-null  object 
 13  medical           61008 non-null  object 
 14  hospital          61008 non-null  object 
 15  openpubl          61008 non-null  object 
 16  year              61008 non-null  int64 

In [5]:
ic_student_join.head(2)

unitid                               instnm  \
0  100654             Alabama A & M University   
1  100663  University Of Alabama At Birmingham   

                             addr        city stabbr          zip     obereg  \
0                4107 MERIDIAN ST      NORMAL     AL      35762.0  Southeast   
1  ADMINISTRATION BLDG SUITE 1070  BIRMINGHAM     AL  352940110.0  Southeast   

          sector             iclevel control  ...  \
0  4-year public  Four or more years  Public  ...   
1  4-year public  Four or more years  Public  ...   

                                      carnegie  \
0  Doctoral or Research Universities-Intensive   
1  Doctoral or Research Universities-Extensive   

                             pset4flg medical hospital  \
0  Title IV postsecondary institution      No       No   
1  Title IV postsecondary institution     Yes      Yes   

                             openpubl  year  academic year  \
0  Insititution is open to the public  2001      2001-2002   
1  Insititution is open to the public  2001      2001-2002   

                    carnegie_grouped  unitid_year student fte  
0  Doctoral or Research Universities  100654_2001      5223.0  
1  Doctoral or Research Universities  100663_2001     11614.0  

[2 rows x 21 columns]

In [6]:
col_names = ic_student_join.columns.values.tolist()
print(col_names)

['unitid', 'instnm', 'addr', 'city', 'stabbr', 'zip', 'obereg', 'sector', 'iclevel', 'control', 'deggrant', 'carnegie', 'pset4flg', 'medical', 'hospital', 'openpubl', 'year', 'academic year', 'carnegie_grouped', 'unitid_year', 'student fte']


Check for missing student enrollment & fte in joined dataframe

1. imputing student enrollment & fte for years institution is reporting to institutional characteristics survey but not the derived Fall student enrollment report

results:
total enrollment          76
total fte student         76
institution size          76

In [7]:
ic_student_join.isnull().sum()
ic_student_join_missing = ic_student_join[ic_student_join['student fte'].isnull()]
#ic_student_join_missing.info()

print(ic_student_join_missing.groupby(['year','carnegie_grouped'])['unitid'].count())


year  carnegie_grouped                       
2001  Associates Colleges                         4
      Baccalaureate Colleges and Universities     1
      Masters Colleges and Universities           3
      Specialized Institutions                    2
2002  Associates Colleges                         4
                                                 ..
2019  Specialized Institutions                   11
2020  Associates Colleges                         2
      Baccalaureate Colleges and Universities     4
      Specialized Institutions                    1
2021  Baccalaureate Colleges and Universities     1
Name: unitid, Length: 74, dtype: int64


In [8]:
ic_student_final = ic_student_join[ic_student_join['student fte'].notna()]

In [9]:
ic_student_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60692 entries, 0 to 61007
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   unitid            60692 non-null  int64  
 1   instnm            60692 non-null  object 
 2   addr              60370 non-null  object 
 3   city              60692 non-null  object 
 4   stabbr            60692 non-null  object 
 5   zip               60692 non-null  object 
 6   obereg            60692 non-null  object 
 7   sector            60692 non-null  object 
 8   iclevel           60692 non-null  object 
 9   control           60692 non-null  object 
 10  deggrant          60692 non-null  object 
 11  carnegie          60692 non-null  object 
 12  pset4flg          60692 non-null  object 
 13  medical           60692 non-null  object 
 14  hospital          60692 non-null  object 
 15  openpubl          60692 non-null  object 
 16  year              60692 non-null  int64 

Adding column for Institutional Size to Institutional Characteristics - Student Enrollment FTE Join

In [10]:

ic_student_final['student fte'] = ic_student_final['student fte'].astype(int)

def institution_size(row):
    if row['student fte'] < 1000:
        return 'Under 1,000'
    elif row['student fte'] >= 1000 and row['student fte'] < 5000:
        return '1,000 - 4,999' 
    elif row['student fte'] >= 5000 and row['student fte'] < 10000:
        return '5,000 - 9,999'   
    elif row['student fte'] >= 10000 and row['student fte'] < 20000:
        return '10,000 - 19,999'   
    elif row['student fte'] >= 20000:
        return '20,000 and above'   


ic_student_final = ic_student_final.copy() 
ic_student_final['institution size'] = ic_student_final.apply(lambda row: institution_size(row), axis=1)

#ic_student_final.head()

/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1905748687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_final['student fte'] = ic_student_final['student fte'].astype(int)


In [11]:
#ic_student_join.to_csv('/Users/ellenruthconnell/Documents/IPEDS/masters_for_merge/ic_student_join.csv',index=False)

In [12]:
ic_student_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60692 entries, 0 to 61007
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   unitid            60692 non-null  int64 
 1   instnm            60692 non-null  object
 2   addr              60370 non-null  object
 3   city              60692 non-null  object
 4   stabbr            60692 non-null  object
 5   zip               60692 non-null  object
 6   obereg            60692 non-null  object
 7   sector            60692 non-null  object
 8   iclevel           60692 non-null  object
 9   control           60692 non-null  object
 10  deggrant          60692 non-null  object
 11  carnegie          60692 non-null  object
 12  pset4flg          60692 non-null  object
 13  medical           60692 non-null  object
 14  hospital          60692 non-null  object
 15  openpubl          60692 non-null  object
 16  year              60692 non-null  int64 
 17  academic yea

Step 2: staff enrollment & fte
1. Upload 
2. Create unique ID for data merging: unitid_year
3. left outer join with ic_master
4. check for missing values

In [13]:
staff_master = pd.read_csv("staff_fte_master_2.csv",low_memory=False,encoding = "ISO-8859-1")
staff_master['unitid_year'] = staff_master['unitid'].astype(str) + "_" + staff_master['year'].astype(str)
#staff_master.head()
staff_master_for_merge = staff_master.drop(columns=['unitid','year'])
staff_master_for_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136916 entries, 0 to 136915
Data columns (total 13 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   total staff                      136916 non-null  float64
 1   total staff ft                   136916 non-null  float64
 2   total staff pt                   136916 non-null  float64
 3   total staff fte                  136916 non-null  float64
 4   total instruction staff          136916 non-null  float64
 5   total instruction ft             136916 non-null  float64
 6   total instruction pt             136916 non-null  float64
 7   total instruction staff fte      136916 non-null  float64
 8   total non instruction            136916 non-null  float64
 9   total non instruction ft         136916 non-null  float64
 10  total non instruction pt         136916 non-null  float64
 11  total non instruction staff fte  136916 non-null  float64
 12  un

Left outer join:
institutional characteristics/ student emrollment master + staff fte 

In [14]:
ic_student_staff_join = pd.merge(ic_student_final,staff_master_for_merge, left_on='unitid_year',right_on='unitid_year',how = 'left')
ic_student_staff_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60692 entries, 0 to 60691
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   unitid                           60692 non-null  int64  
 1   instnm                           60692 non-null  object 
 2   addr                             60370 non-null  object 
 3   city                             60692 non-null  object 
 4   stabbr                           60692 non-null  object 
 5   zip                              60692 non-null  object 
 6   obereg                           60692 non-null  object 
 7   sector                           60692 non-null  object 
 8   iclevel                          60692 non-null  object 
 9   control                          60692 non-null  object 
 10  deggrant                         60692 non-null  object 
 11  carnegie                         60692 non-null  object 
 12  pset4flg          

In [15]:
staff_missing = ic_student_staff_join[ic_student_staff_join['total staff fte'].isna()]
staff_missing_list = staff_missing['unitid_year'].tolist()

ic_student_staff_join_na_dropped = ic_student_staff_join[~ic_student_staff_join['unitid_year'].isin(staff_missing_list)]
#print(staff_missing.groupby('year')['unitid'].count())

In [16]:
ic_student_staff_join_na_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57607 entries, 3009 to 60691
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   unitid                           57607 non-null  int64  
 1   instnm                           57607 non-null  object 
 2   addr                             57372 non-null  object 
 3   city                             57607 non-null  object 
 4   stabbr                           57607 non-null  object 
 5   zip                              57607 non-null  object 
 6   obereg                           57607 non-null  object 
 7   sector                           57607 non-null  object 
 8   iclevel                          57607 non-null  object 
 9   control                          57607 non-null  object 
 10  deggrant                         57607 non-null  object 
 11  carnegie                         57607 non-null  object 
 12  pset4flg       

In [17]:
ic_student_staff_master = ic_student_staff_join_na_dropped

In [18]:
def student_staff_ratio(row):
    if row['total staff fte'] == 0:
        return 0
    else:
        return (row['student fte']/row['total staff fte'])
    
ic_student_staff_master['student fte per staff fte'] = ic_student_staff_master.apply(lambda row: student_staff_ratio(row), axis=1)
ic_student_staff_master['student fte per staff fte'] = ic_student_staff_master['student fte per staff fte'].round(2)

ic_student_staff_master.head()

/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1453248184.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_staff_master['student fte per staff fte'] = ic_student_staff_master.apply(lambda row: student_staff_ratio(row), axis=1)
/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1453248184.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_staff_master['student fte per staff fte'] = ic_student_staff_master['student fte per staff fte'].round(2)


unitid                               instnm  \
3009  100654             Alabama A & M University   
3010  100663  University Of Alabama At Birmingham   
3011  100690                   Amridge University   
3012  100706  University Of Alabama In Huntsville   
3013  100724             Alabama State University   

                                addr        city stabbr         zip  \
3009                4107 MERIDIAN ST      NORMAL     AL       35762   
3010  ADMINISTRATION BLDG SUITE 1070  BIRMINGHAM     AL  35294-0110   
3011                  1200 TAYLOR RD  MONTGOMERY     AL  36117-3553   
3012                 301 SPARKMAN DR  HUNTSVILLE     AL       35899   
3013                915 S JACKSON ST  MONTGOMERY     AL  36101-0271   

         obereg                          sector             iclevel  \
3009  Southeast                   4-year public  Four or more years   
3010  Southeast                   4-year public  Four or more years   
3011  Southeast  4-year private, not-for-profit  Four or more years   
3012  Southeast                   4-year public  Four or more years   
3013  Southeast                   4-year public  Four or more years   

                     control  ... total staff fte total instruction staff  \
3009                  Public  ...          1082.0                   376.0   
3010                  Public  ...          7747.0                  2925.0   
3011  Private not-for-profit  ...            42.0                    10.0   
3012                  Public  ...          1109.0                   474.0   
3013                  Public  ...           908.0                   381.0   

     total instruction ft total instruction pt total instruction staff fte  \
3009                292.0                 84.0                       320.0   
3010               2648.0                277.0                      2740.0   
3011                 10.0                  0.0                        10.0   
3012                287.0                187.0                       349.0   
3013                231.0                150.0                       281.0   

     total non instruction  total non instruction ft total non instruction pt  \
3009                 842.0                     722.0                    120.0   
3010                5879.0                    4572.0                   1307.0   
3011                  33.0                      32.0                      1.0   
3012                 768.0                     755.0                     13.0   
3013                 696.0                     593.0                    103.0   

     total non instruction staff fte student fte per staff fte  
3009                           762.0                      4.86  
3010                          5007.0                      1.59  
3011                            32.0                     10.93  
3012                           759.0                      4.76  
3013                           627.0                      5.75  

[5 rows x 35 columns]

In [19]:
def student_instruction_staff_ratio(row):
    if row['total instruction staff fte'] == 0:
        return 0
    else:
        return (row['student fte']/row['total instruction staff fte'])
    
ic_student_staff_master['student fte per instruction staff fte'] = ic_student_staff_master.apply(lambda row: student_instruction_staff_ratio(row), axis=1)
ic_student_staff_master['student fte per instruction staff fte'] = ic_student_staff_master['student fte per instruction staff fte'].round(2)

ic_student_staff_master.head()

/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1536001270.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_staff_master['student fte per instruction staff fte'] = ic_student_staff_master.apply(lambda row: student_instruction_staff_ratio(row), axis=1)
/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1536001270.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_staff_master['student fte per instruction staff fte'] = ic_student_staff_master['student f

unitid                               instnm  \
3009  100654             Alabama A & M University   
3010  100663  University Of Alabama At Birmingham   
3011  100690                   Amridge University   
3012  100706  University Of Alabama In Huntsville   
3013  100724             Alabama State University   

                                addr        city stabbr         zip  \
3009                4107 MERIDIAN ST      NORMAL     AL       35762   
3010  ADMINISTRATION BLDG SUITE 1070  BIRMINGHAM     AL  35294-0110   
3011                  1200 TAYLOR RD  MONTGOMERY     AL  36117-3553   
3012                 301 SPARKMAN DR  HUNTSVILLE     AL       35899   
3013                915 S JACKSON ST  MONTGOMERY     AL  36101-0271   

         obereg                          sector             iclevel  \
3009  Southeast                   4-year public  Four or more years   
3010  Southeast                   4-year public  Four or more years   
3011  Southeast  4-year private, not-for-profit  Four or more years   
3012  Southeast                   4-year public  Four or more years   
3013  Southeast                   4-year public  Four or more years   

                     control  ... total instruction staff  \
3009                  Public  ...                   376.0   
3010                  Public  ...                  2925.0   
3011  Private not-for-profit  ...                    10.0   
3012                  Public  ...                   474.0   
3013                  Public  ...                   381.0   

     total instruction ft total instruction pt total instruction staff fte  \
3009                292.0                 84.0                       320.0   
3010               2648.0                277.0                      2740.0   
3011                 10.0                  0.0                        10.0   
3012                287.0                187.0                       349.0   
3013                231.0                150.0                       281.0   

     total non instruction total non instruction ft  total non instruction pt  \
3009                 842.0                    722.0                     120.0   
3010                5879.0                   4572.0                    1307.0   
3011                  33.0                     32.0                       1.0   
3012                 768.0                    755.0                      13.0   
3013                 696.0                    593.0                     103.0   

     total non instruction staff fte student fte per staff fte  \
3009                           762.0                      4.86   
3010                          5007.0                      1.59   
3011                            32.0                     10.93   
3012                           759.0                      4.76   
3013                           627.0                      5.75   

     student fte per instruction staff fte  
3009                                 16.44  
3010                                  4.51  
3011                                 45.90  
3012                                 15.13  
3013                                 18.58  

[5 rows x 36 columns]

In [20]:
def student_non_instruction_staff_ratio(row):
    if row['total non instruction staff fte'] == 0:
        return 0
    else:
        return (row['student fte']/row['total non instruction staff fte'])
    
ic_student_staff_master['student fte per non instruction staff fte'] = ic_student_staff_master.apply(lambda row: student_non_instruction_staff_ratio(row), axis=1)
ic_student_staff_master['student fte per non instruction staff fte'] = ic_student_staff_master['student fte per non instruction staff fte'].round(2)

ic_student_staff_master.head()

/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1123147392.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_staff_master['student fte per non instruction staff fte'] = ic_student_staff_master.apply(lambda row: student_non_instruction_staff_ratio(row), axis=1)
/var/folders/5l/g6jpvcjs5mddr5zrcyyl0y4w0000gn/T/ipykernel_50913/1123147392.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ic_student_staff_master['student fte per non instruction staff fte'] = ic_student_staff_maste

unitid                               instnm  \
3009  100654             Alabama A & M University   
3010  100663  University Of Alabama At Birmingham   
3011  100690                   Amridge University   
3012  100706  University Of Alabama In Huntsville   
3013  100724             Alabama State University   

                                addr        city stabbr         zip  \
3009                4107 MERIDIAN ST      NORMAL     AL       35762   
3010  ADMINISTRATION BLDG SUITE 1070  BIRMINGHAM     AL  35294-0110   
3011                  1200 TAYLOR RD  MONTGOMERY     AL  36117-3553   
3012                 301 SPARKMAN DR  HUNTSVILLE     AL       35899   
3013                915 S JACKSON ST  MONTGOMERY     AL  36101-0271   

         obereg                          sector             iclevel  \
3009  Southeast                   4-year public  Four or more years   
3010  Southeast                   4-year public  Four or more years   
3011  Southeast  4-year private, not-for-profit  Four or more years   
3012  Southeast                   4-year public  Four or more years   
3013  Southeast                   4-year public  Four or more years   

                     control  ... total instruction ft total instruction pt  \
3009                  Public  ...                292.0                 84.0   
3010                  Public  ...               2648.0                277.0   
3011  Private not-for-profit  ...                 10.0                  0.0   
3012                  Public  ...                287.0                187.0   
3013                  Public  ...                231.0                150.0   

     total instruction staff fte total non instruction  \
3009                       320.0                 842.0   
3010                      2740.0                5879.0   
3011                        10.0                  33.0   
3012                       349.0                 768.0   
3013                       281.0                 696.0   

     total non instruction ft total non instruction pt  \
3009                    722.0                    120.0   
3010                   4572.0                   1307.0   
3011                     32.0                      1.0   
3012                    755.0                     13.0   
3013                    593.0                    103.0   

      total non instruction staff fte student fte per staff fte  \
3009                            762.0                      4.86   
3010                           5007.0                      1.59   
3011                             32.0                     10.93   
3012                            759.0                      4.76   
3013                            627.0                      5.75   

     student fte per instruction staff fte  \
3009                                 16.44   
3010                                  4.51   
3011                                 45.90   
3012                                 15.13   
3013                                 18.58   

     student fte per non instruction staff fte  
3009                                      6.90  
3010                                      2.47  
3011                                     14.34  
3012                                      6.96  
3013                                      8.33  

[5 rows x 37 columns]

In [21]:
ic_student_staff_master_check = ic_student_staff_master[ic_student_staff_master['student fte per staff fte']==0]

#print(ic_student_staff_master_check.groupby('year')['unitid'].nunique())

ic_student_staff_master_check_list = ic_student_staff_master_check['unitid_year'].tolist()

ic_student_staff_master = ic_student_staff_master[~ic_student_staff_master['unitid_year'].isin(ic_student_staff_master_check_list)]

In [22]:
ic_student_staff_master['student fte per staff fte'].describe()

count    57606.000000
mean         7.253397
std         16.604913
min          0.060000
25%          4.730000
50%          6.470000
75%          8.930000
max       3870.420000
Name: student fte per staff fte, dtype: float64

In [23]:
Q1 = ic_student_staff_master['student fte per staff fte'].quantile(0.25)
Q3 = ic_student_staff_master['student fte per staff fte'].quantile(0.75)
iqr = Q3 - Q1

upper = Q3 + (1.5 * iqr)

outlier = Q3 + (4 * iqr)

#ic_student_staff_master_upper = ic_student_staff_master[ic_student_staff_master['student fte per staff fte']>upper]
#print(ic_student_staff_master_upper.groupby('carnegie_grouped').agg({'unitid':'nunique','student fte per staff fte':'min','student fte per staff fte':'max'}))

ic_student_staff_master_outlier = ic_student_staff_master[ic_student_staff_master['student fte per staff fte']>outlier]
print(ic_student_staff_master_outlier.groupby('carnegie_grouped').agg({'unitid':'nunique','student fte per staff fte':'min'}))

ic_student_staff_master_outlier_list = ic_student_staff_master_outlier['unitid_year'].tolist()
ic_student_staff_master = ic_student_staff_master[~ic_student_staff_master['unitid_year'].isin(ic_student_staff_master_outlier_list)]


                                         unitid  student fte per staff fte
carnegie_grouped                                                          
Associates Colleges                          35                      25.81
Baccalaureate Colleges and Universities       3                      26.17
Masters Colleges and Universities             8                      26.89
Specialized Institutions                     23                      25.75


In [24]:
print(ic_student_staff_master['student fte per staff fte'].describe())

count    57463.000000
mean         7.100354
std          3.549899
min          0.060000
25%          4.720000
50%          6.460000
75%          8.900000
max         25.700000
Name: student fte per staff fte, dtype: float64


In [25]:
Q1_instruction = ic_student_staff_master['student fte per instruction staff fte'].quantile(0.25)
Q3_instruction = ic_student_staff_master['student fte per instruction staff fte'].quantile(0.75)
iqr_instruction = Q3_instruction - Q1_instruction

upper_instruction = Q3_instruction + (1.5 * iqr_instruction)

outlier_instruction = Q3_instruction + (3 * iqr_instruction)

ic_student_staff_master['student fte per instruction staff fte'].describe()

ic_student_staff_master_outlier_instruction = ic_student_staff_master[ic_student_staff_master['student fte per instruction staff fte']>outlier_instruction]
print(ic_student_staff_master_outlier_instruction.groupby('carnegie_grouped').agg({'unitid':'nunique','student fte per instruction staff fte':'min'}))

ic_student_staff_master_outlier_instruction_list = ic_student_staff_master_outlier_instruction['unitid_year'].tolist()
ic_student_staff_master = ic_student_staff_master[~ic_student_staff_master['unitid_year'].isin(ic_student_staff_master_outlier_instruction_list)]

                                         unitid  \
carnegie_grouped                                  
Associates Colleges                          43   
Baccalaureate Colleges and Universities       8   
Masters Colleges and Universities            16   
Specialized Institutions                     39   

                                         student fte per instruction staff fte  
carnegie_grouped                                                                
Associates Colleges                                                      45.98  
Baccalaureate Colleges and Universities                                  47.09  
Masters Colleges and Universities                                        45.98  
Specialized Institutions                                                 46.00  


In [26]:
Q1_non_instruction = ic_student_staff_master['student fte per non instruction staff fte'].quantile(0.25)
Q3_non_instruction = ic_student_staff_master['student fte per non instruction staff fte'].quantile(0.75)
iqr_non_instruction = Q3_non_instruction - Q1_non_instruction

upper_non_instruction = Q3_non_instruction + (1.5 * iqr_non_instruction)

outlier_non_instruction = Q3_non_instruction + (3 * iqr_non_instruction)

ic_student_staff_master['student fte per non instruction staff fte'].describe()

ic_student_staff_master_outlier_non_instruction = ic_student_staff_master[ic_student_staff_master['student fte per non instruction staff fte']>outlier_non_instruction]
print(ic_student_staff_master_outlier_non_instruction.groupby('carnegie_grouped').agg({'unitid':'nunique','student fte per non instruction staff fte':'min'}))

ic_student_staff_master_outlier_non_instruction_list = ic_student_staff_master_outlier_non_instruction['unitid_year'].tolist()
ic_student_staff_master = ic_student_staff_master[~ic_student_staff_master['unitid_year'].isin(ic_student_staff_master_outlier_non_instruction_list)]

ic_student_staff_master = ic_student_staff_master.reset_index(drop=True)

                                         unitid  \
carnegie_grouped                                  
Associates Colleges                         114   
Baccalaureate Colleges and Universities       9   
Doctoral or Research Universities             2   
Masters Colleges and Universities             9   
Specialized Institutions                     46   

                                         student fte per non instruction staff fte  
carnegie_grouped                                                                    
Associates Colleges                                                          43.09  
Baccalaureate Colleges and Universities                                      43.12  
Doctoral or Research Universities                                            44.60  
Masters Colleges and Universities                                            43.44  
Specialized Institutions                                                     43.28  


In [27]:
ic_student_staff_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56353 entries, 0 to 56352
Data columns (total 37 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   unitid                                     56353 non-null  int64  
 1   instnm                                     56353 non-null  object 
 2   addr                                       56118 non-null  object 
 3   city                                       56353 non-null  object 
 4   stabbr                                     56353 non-null  object 
 5   zip                                        56353 non-null  object 
 6   obereg                                     56353 non-null  object 
 7   sector                                     56353 non-null  object 
 8   iclevel                                    56353 non-null  object 
 9   control                                    56353 non-null  object 
 10  deggrant              

In [28]:
ic_student_staff_master['student fte per non instruction staff fte'].describe()

count    56353.000000
mean        12.589690
std          7.374275
min          0.000000
25%          7.510000
50%         11.000000
75%         16.120000
max         43.070000
Name: student fte per non instruction staff fte, dtype: float64

In [29]:
ic_student_staff_master['student fte per instruction staff fte'].describe()

count    56353.000000
mean        16.528471
std          6.575965
min          0.000000
25%         12.140000
50%         15.910000
75%         20.350000
max         45.930000
Name: student fte per instruction staff fte, dtype: float64

I need to confirm next that I have all of the Ivy's and Yale in the dataset

130794, Yale

In [30]:
ivy_list = [130794,166027,182670,186131,190150,190415,215062,217156]

def ivy(row):
    if row['unitid'] in ivy_list:
        return 'ivy'
    else:
        return 'not an ivy'
    
ic_student_staff_master['ivy league'] = ic_student_staff_master.apply(lambda row: ivy(row), axis=1)

ic_student_staff_master.head()

unitid                               instnm  \
0  100654             Alabama A & M University   
1  100663  University Of Alabama At Birmingham   
2  100690                   Amridge University   
3  100706  University Of Alabama In Huntsville   
4  100724             Alabama State University   

                             addr        city stabbr         zip     obereg  \
0                4107 MERIDIAN ST      NORMAL     AL       35762  Southeast   
1  ADMINISTRATION BLDG SUITE 1070  BIRMINGHAM     AL  35294-0110  Southeast   
2                  1200 TAYLOR RD  MONTGOMERY     AL  36117-3553  Southeast   
3                 301 SPARKMAN DR  HUNTSVILLE     AL       35899  Southeast   
4                915 S JACKSON ST  MONTGOMERY     AL  36101-0271  Southeast   

                           sector             iclevel                 control  \
0                   4-year public  Four or more years                  Public   
1                   4-year public  Four or more years                  Public   
2  4-year private, not-for-profit  Four or more years  Private not-for-profit   
3                   4-year public  Four or more years                  Public   
4                   4-year public  Four or more years                  Public   

   ... total instruction pt total instruction staff fte total non instruction  \
0  ...                 84.0                       320.0                 842.0   
1  ...                277.0                      2740.0                5879.0   
2  ...                  0.0                        10.0                  33.0   
3  ...                187.0                       349.0                 768.0   
4  ...                150.0                       281.0                 696.0   

  total non instruction ft total non instruction pt  \
0                    722.0                    120.0   
1                   4572.0                   1307.0   
2                     32.0                      1.0   
3                    755.0                     13.0   
4                    593.0                    103.0   

  total non instruction staff fte  student fte per staff fte  \
0                           762.0                       4.86   
1                          5007.0                       1.59   
2                            32.0                      10.93   
3                           759.0                       4.76   
4                           627.0                       5.75   

  student fte per instruction staff fte  \
0                                 16.44   
1                                  4.51   
2                                 45.90   
3                                 15.13   
4                                 18.58   

  student fte per non instruction staff fte  ivy league  
0                                      6.90  not an ivy  
1                                      2.47  not an ivy  
2                                     14.34  not an ivy  
3                                      6.96  not an ivy  
4                                      8.33  not an ivy  

[5 rows x 38 columns]

In [31]:
ivy_test = ic_student_staff_master.groupby(['year','ivy league']).agg({'unitid':['count']})
print(ivy_test)

                unitid
                 count
year ivy league       
2002 ivy             8
     not an ivy   2908
2003 ivy             8
     not an ivy   2872
2004 ivy             8
     not an ivy   2906
2005 ivy             8
     not an ivy   2895
2006 ivy             8
     not an ivy   2897
2007 ivy             8
     not an ivy   2886
2008 ivy             8
     not an ivy   2864
2009 ivy             8
     not an ivy   2825
2010 ivy             8
     not an ivy   2814
2011 ivy             8
     not an ivy   2813
2012 ivy             8
     not an ivy   2801
2013 ivy             8
     not an ivy   2797
2014 ivy             8
     not an ivy   2770
2015 ivy             8
     not an ivy   2774
2016 ivy             8
     not an ivy   2754
2017 ivy             8
     not an ivy   2748
2018 ivy             8
     not an ivy   2739
2019 ivy             8
     not an ivy   2712
2020 ivy             8
     not an ivy   2702
2021 ivy             8
     not an ivy   2716


In [32]:
ic_student_staff_master.to_csv('/Users/ellenruthconnell/Documents/INFO_696/3_eda/ic_student_staff_master_incl_medical.csv',index=False)